In [91]:
#------------------------------importing library---------------------------------------
from dateutil.parser import parse
import numpy as np
import pandas as pd
import pickle
# from bpemb import BPEmb
import pytesseract
import cv2
import glob
%matplotlib notebook
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
import re
from timeit import default_timer as timer


from xml.etree import ElementTree
from xml.dom import minidom
import xml.etree.cElementTree as ET 

In [92]:
text_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.txt"
img_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.jpg"
tesseract_path = 'C:/python_lib/ocr/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = 'C:/python_lib/ocr/tesseract.exe'
file_save_path='C:/Users/Sanjeev/Documents/Python Scripts/Dataset/label_data/'
xml_file="C:/Users/Sanjeev/Documents/Python Scripts/NLP/TensorFlow/addons/data/X00016469612.xml"

In [238]:
text_path_list = glob.glob(text_path)
img_path_list = glob.glob(img_path)

In [253]:
def SaveData(data,path):
    with open(path,'wb') as file:
        pickle.dump(data,file)

def LoadData(path):
    with open(path, 'rb') as file:
        return pickle.load(file)

def display_image(img):
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    if len(img.shape) > 2:
        plt.imshow(img, cmap='gray')
        plt.title("Image")
        plt.axis('off')
    else:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title("RGB Image")
        plt.axis('off')

        
def extract_bbox_info(img_path):
    hocr_data = pytesseract.image_to_pdf_or_hocr(img_path, extension='hocr')
    soup = BeautifulSoup(hocr_data)
    spans = soup.find_all('span', attrs={'class':'ocrx_word'})  # getting the attributes 
    word_bbox_list =[]  
#     bbox_width=[]
    for span in spans:
        bbox_str=span['title']  # geting bounding box inforamtion 
        bbox_str =bbox_str.split(';')
        bbox =bbox_str[0].split()[1:]
    #     x,y,w,h= *bbox
    #     word_info_list.append([span['id'],span.string,[bbox[1].split()[-1],*bbox]])
        word_bbox_list.append([span['id'],span.string,
                               [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                              ])
#         bbox_width.append((int(bbox[2])-int(bbox[0])))
#         word_info_list.sort(key = lambda sort_value: (sort_value[2][1],sort_value[2][0]))
#     word_info_list.sort(key = lambda sort_value: sort_value[2][1])
    
    return word_bbox_list

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ") 
# from ElementTree_pretty import prettify
def create_xml(bbox_list=None,path=None,img_shape=None,folder='Data'):
    w,h,d=img_shape
    
    root = ET.Element("annotation")
    root_c_folder = ET.SubElement(root, "folder").text =folder
    root_c_filename=ET.SubElement(root, "filename").text =path.split('\\')[-1]
    
    root_c_path=ET.SubElement(root, "path").text =path
    root_c_source=ET.SubElement(root, "source")
    ET.SubElement(root_c_source, "database").text = "Unknown"
    
    root_c_size=ET.SubElement(root, "size")
    ET.SubElement(root_c_size, "width").text = str(int(w))
    ET.SubElement(root_c_size, "height").text = str(int(h))
    ET.SubElement(root_c_size, "depth").text = str(int(d))
    
    root_c_segmented=ET.SubElement(root, "segmented").text=str(int(0))
    
    for bbox_ in bbox_list:
        bbox =bbox_[-1]
        root_c_object=ET.SubElement(root, "object")

        ET.SubElement(root_c_object, "name").text = 'text'
        ET.SubElement(root_c_object, "pose").text = 'Unspecified'
        ET.SubElement(root_c_object, "truncated").text = str(int(0))
        ET.SubElement(root_c_object, "difficult").text = str(int(0))
        object_c_bndbox=ET.SubElement(root_c_object, "bndbox")
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[0]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[1]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[2]))
        ET.SubElement(object_c_bndbox, "xmin").text = str(int(bbox[3]))

    return prettify(root)
#     tree.write(file_save_path+"filename.xml")

In [247]:
def get_ocr_text(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
#     print(text)
    return text

# read_data ()-> out data [ocr_text_list,file_name_list,word_list,dict_list,img_list,path_list]
def read_data(img_path_list, text_path_list, data_no=150):
    word_list,dict_list,img_list,path_list,file_name_list,ocr_text_list=[],[],[],[],[],[]
    j=0
    for i in range(data_no):
        img_path= img_path_list[i]
        path_list.append(img_path_list[i])
        img_name = img_path.split("\\")[-1]
        img_name = img_name.split(".jpg")[0]
        img_data = cv2.imread(img_path_list[i])
        img_list.append(img_data)
        ocr_text = get_ocr_text(img_path)
        ocr_text_list.append(ocr_text)
        word_list.append(extract_bbox_info(img_path_list[i]))
        text_path = text_path_list[j]
        text_name=text_path.split("\\")[-1]
        text_name = text_name.split('.txt')[0]
        while img_name != text_name:
            j +=1
            print('fileName is not same: ',img_name, text_name)
            text_path = text_path_list[j]
            text_name=text_path.split("\\")[-1]
            text_name = text_name.split('.txt')[0]
        with open(text_path, 'r') as file:
            text_data =file.read()
        dict_list.append(eval(text_data))
        file_name_list.append(img_name)
#         data_list.append([img_data,ocr_text,eval(text_data),img_name])    
    return ocr_text_list,file_name_list,word_list,dict_list,img_list,path_list

In [237]:
def Line_Formation(word_list):
    word_list.sort(key =lambda sort_on_top : sort_on_top[2][1]) # sorting on top value 
    line_list =[]  
    word_list_len = len(word_list)
    temp=[]
    temp.append(word_list[0])   # intializing the value with pre-assumption 
    for i in range(1,word_list_len, 1):
        wa_top =word_list[i-1][2][1]
        wa_bottom = word_list[i-1][2][3]

        wb_top =word_list[i][2][1]
        wb_bottom = word_list[i][2][3]
        if wa_top <= wb_bottom and wa_bottom >= wb_top:
            temp.append(word_list[i])
        else:
            temp.sort(key=lambda sort_on_left_ : sort_on_left_[2][0]) # sorting on the left value
            line_list.append(temp)
            temp=[]
            temp.append(word_list[i])
    line_list.append(temp)
    return line_list

# output-> arr_word_list, dict_list
def create_output(word_list,dict_word):
    dict_list = []
    for value in dict_word.values():
        dict_list.append(value.split())

        arr_word_list=[]
    lines=Line_Formation(word_list)
    for line in lines:
        for word in line:
            arr_word_list.append(word)
        
    return arr_word_list, dict_list

In [244]:
def get_index(text,dict_list):
    for idx,word_list in enumerate(dict_list):
        if text in word_list:
            return idx
    return -1

def get_bbox_list(arr_word_list,dict_list):
    bbox_list,idx =[],0
    while idx < len(arr_word_list):
        text= arr_word_list[idx][1]
        value_idx = get_index(text,dict_list)
        if value_idx ==-1:
            idx +=1
            continue
#         print('value_id : ',value_idx,'idx ', idx, 'text = ', text)
#         print(text,"--->", dict_list[value_idx])
        if text in dict_list[value_idx]:     #case when 1st word matches
            l=dict_list[value_idx].index(text) # if index is not zero
#             print('-------> index check (',text,'): ', l )
            j=0
            if l != 0:
#                 print('case when l not eaual to zero vlaue l ', l)
                j=l
                while l > 0:
#                     print('appending word: ',arr_word_list[idx-l])
                    bbox_list.append(arr_word_list[idx-l])
                    l=l-1
            while j < len(dict_list[value_idx]) and arr_word_list[idx][1] != dict_list[value_idx][-1]:  
                bbox_list.append(arr_word_list[idx])  # todo what should be the output data actural data or extracted one (now). 
                idx +=1
                j +=1
            bbox_list.append(arr_word_list[idx])
            del dict_list[value_idx]
            value_idx +=1
        idx +=1
    return bbox_list
    

In [260]:
_,_,word_list,dict_list,img_list,path_list = read_data(img_path_list, text_path_list, data_no=2)

reading image ../n
reading image ../n
fileName is not same:  X00016469619 X00016469612


In [277]:
def create_xml_annotation(word_list, dict_list, img_list, path_list, file_save_path=''):
    
    file_len =len(word_list)
    
    for idx in range(file_len):
        
        arr_word_list, dict_text_list = create_output(word_list[idx],dict_list[idx])
        bbox_list = get_bbox_list(arr_word_list,dict_text_list)

        xml_text=create_xml(bbox_list,path_list[idx],img_list[idx].shape,folder='Data')
        file_name = path_list[idx].split('\\')[-1]
        file_name=file_name.split('.')[0]+'.xml'
        with open(file_save_path+file_name, "w") as file:
            file.write(xml_text)
#         print(xml_text,'\n\n---------------------------------\n\n')

In [278]:
create_xml_annotation(word_list, dict_list, img_list, path_list,file_save_path)

In [234]:

arr_word_list,dict_list = create_out(d[2][0],d[3][0])
print(dict_list)
print([word[1] for word in arr_word_list])

[['BOOK', 'TA', '.K', '(TAMAN', 'DAYA)', 'SDN', 'BHD'], ['25/12/2018'], ['NO.53', '55,57', '&', '59,', 'JALAN', 'SAGU', '18,', 'TAMAN', 'DAYA,', '81100', 'JOHOR', 'BAHRU,', 'JOHOR.'], ['9.00']]
['tan', 'woon', 'yann', 'BOOK', 'TA-K', '(TAMAN', 'DAYA)', 'SDN', 'BHD', 'B94', '7-W', 'NO.5?', '55,57', '&', '59,', 'JALAN', 'SAGU', '18,', 'TAMAN', 'DAYA', '81100', 'JOHOR', 'BAHRU,', 'JOHOR.', 'LAM', 'MCU', 'Document', 'Ho', ':', 'TDO1167104', 'Date', '25/12/2018', '8:13:39', 'PM', 'Cashier', 'MANIS', 'Member', 'CASH', 'BILL', 'CODE/DESC', 'PRICE', '—', 'Disc', 'AMOUIT', 'Quy', 'RM', 'RM', '9556939040118', 'KF', 'MODELLING', 'CLAY', 'KIDDY', ' ', 'FISH', '1PC', '*', '9.00)', '6,00', '9.00', 'Total', ':', '9.00', 'Rour', 'ding', 'Adjustment', '0.00', 'Round:', ':d', 'Total', '(RM):', '9.60', 'Cash', ' ', 'CHANGE', 'GOODS', 'SOLD', 'ARE', 'NOT', 'RETURNAR', 'EXCHANGEABLE', ' ', 'THANK', 'YOU.', 'PLEASE', 'COME', 'AGATY', 't']


In [235]:
get_bbox_list(arr_word_list,dict_list)

value_id :  0 idx  3 text =  BOOK
BOOK ---> ['BOOK', 'TA', '.K', '(TAMAN', 'DAYA)', 'SDN', 'BHD']
-------> index check ( BOOK ):  0
value_id :  1 idx  12 text =  55,57
55,57 ---> ['NO.53', '55,57', '&', '59,', 'JALAN', 'SAGU', '18,', 'TAMAN', 'DAYA,', '81100', 'JOHOR', 'BAHRU,', 'JOHOR.']
-------> index check ( 55,57 ):  1
case when l not eaual to zero vlaue l  1
appending word:  ['word_1_12', 'NO.5?', [112, 147, 155, 161]]
value_id :  0 idx  31 text =  25/12/2018
25/12/2018 ---> ['25/12/2018']
-------> index check ( 25/12/2018 ):  0
value_id :  0 idx  58 text =  9.00
9.00 ---> ['9.00']
-------> index check ( 9.00 ):  0


[['word_1_4', 'BOOK', [73, 97, 125, 110]],
 ['word_1_5', 'TA-K', [132, 97, 178, 110]],
 ['word_1_6', '(TAMAN', [186, 95, 261, 112]],
 ['word_1_7', 'DAYA)', [269, 95, 327, 112]],
 ['word_1_8', 'SDN', [335, 96, 370, 109]],
 ['word_1_9', 'BHD', [380, 96, 419, 109]],
 ['word_1_12', 'NO.5?', [112, 147, 155, 161]],
 ['word_1_13', '55,57', [165, 147, 204, 163]],
 ['word_1_14', '&', [211, 147, 221, 159]],
 ['word_1_15', '59,', [229, 147, 248, 162]],
 ['word_1_16', 'JALAN', [260, 147, 303, 160]],
 ['word_1_17', 'SAGU', [311, 146, 353, 159]],
 ['word_1_18', '18,', [364, 146, 381, 162]],
 ['word_1_19', 'TAMAN', [196, 170, 247, 184]],
 ['word_1_20', 'DAYA', [255, 170, 295, 184]],
 ['word_1_21', '81100', [164, 195, 209, 208]],
 ['word_1_22', 'JOHOR', [215, 194, 268, 207]],
 ['word_1_23', 'BAHRU,', [276, 194, 329, 208]],
 ['word_1_24', 'JOHOR.', [219, 217, 271, 231]],
 ['word_1_33', '25/12/2018', [165, 373, 250, 389]],
 ['word_1_59', '9.00', [412, 599, 443, 610]]]

In [23]:
d=read_data(img_file,text_file,1)

reading image ../n


In [107]:
print(d[1][0].split())

print([text[1] for text in d[2][0]])

# display_image(d[0][0])

['tan', 'woon', 'yann', 'BOOK', 'TAK', '(TAMAN', 'DAYA)', 'SDN', 'BHD', 'B97', 'NO.5?', '55,57', '&', '59,', 'JALAN', 'SAGU', '18,', 'TAMAN', 'DAYA', '81100', 'JOHOR', 'BAHRU,', 'JOHOR.', 'WAM', 'MICA', 'A', 'Document', 'Ho', ':', 'TDO1167104', 'Date', '25/12/2018', '8:13:39', 'PM', 'Cashier', 'MANIS', 'Member', 'CASH', 'BILL', 'CODE/DESC', 'PRICE', 'Disc', 'AMOUITT', 'Quy', 'RM', 'RM', '9556939040118', 'KF', 'MODELLING', 'CLAY', 'KIDDY', 'FISH', '1PC', '*', '9.00)', '6,00', '9.00', 'Total', ':', '9,00', 'Rour', 'ding', 'Adjustment', '0.00', 'Round::d', 'Total', '(RM):', '9.60', 'Cash', 'CHANGE', 'GOODS', 'SOLD', 'ARE', 'NOT', 'RETURNAR', 'EXCHANGEABLE', 'THANK', 'YOU.', 'PLEASE', 'COME', 'AGA', 't']
['tan', 'woon', 'yann', 'BOOK', 'TA-K', '(TAMAN', 'DAYA)', 'SDN', 'BHD', 'B94', '7-W', 'NO.5?', '55,57', '&', '59,', 'JALAN', 'SAGU', '18,', 'TAMAN', 'DAYA', '81100', 'JOHOR', 'BAHRU,', 'JOHOR.', 'LAM', 'MCU', 'Document', 'Ho', ':', 'TDO1167104', ' ', 'Date', '25/12/2018', '8:13:39', 'PM',

In [266]:
xml_f="C:/Users/Sanjeev/Documents/Python Scripts/Dataset/label_data/X00016469612.xml"

In [267]:
from xml.dom import minidom

# parse an xml file by name
mydoc = minidom.parse(xml_f)

items = mydoc.getElementsByTagName('object')

In [268]:
items

[<DOM Element: object at 0x163b766acc8>,
 <DOM Element: object at 0x163b7672368>,
 <DOM Element: object at 0x163b76729a8>,
 <DOM Element: object at 0x163b767a048>,
 <DOM Element: object at 0x163b767a688>,
 <DOM Element: object at 0x163b767acc8>,
 <DOM Element: object at 0x163b767b368>,
 <DOM Element: object at 0x163b767b9a8>,
 <DOM Element: object at 0x163b767f048>,
 <DOM Element: object at 0x163b767f688>,
 <DOM Element: object at 0x163b767fcc8>,
 <DOM Element: object at 0x163b7683368>,
 <DOM Element: object at 0x163b76839a8>,
 <DOM Element: object at 0x163b7676048>,
 <DOM Element: object at 0x163b7676688>,
 <DOM Element: object at 0x163b7676cc8>,
 <DOM Element: object at 0x163b7688368>,
 <DOM Element: object at 0x163b76889a8>,
 <DOM Element: object at 0x163b768b048>,
 <DOM Element: object at 0x163b768b688>,
 <DOM Element: object at 0x163b768bcc8>]

In [276]:
import xml.etree.ElementTree as ET
tree = ET.parse(xml_f)
root = tree.getroot()
print('\nAll item data:')
for elem in root:
    for subelem in elem:
        print(subelem.text)
        for bbox in subelem:
            print('bbox :-> ',bbox.text)



All item data:
Unknown
1013
463
3
text
Unspecified
0
0

      
bbox :->  73
bbox :->  97
bbox :->  125
bbox :->  110
text
Unspecified
0
0

      
bbox :->  132
bbox :->  97
bbox :->  178
bbox :->  110
text
Unspecified
0
0

      
bbox :->  186
bbox :->  95
bbox :->  261
bbox :->  112
text
Unspecified
0
0

      
bbox :->  269
bbox :->  95
bbox :->  327
bbox :->  112
text
Unspecified
0
0

      
bbox :->  335
bbox :->  96
bbox :->  370
bbox :->  109
text
Unspecified
0
0

      
bbox :->  380
bbox :->  96
bbox :->  419
bbox :->  109
text
Unspecified
0
0

      
bbox :->  112
bbox :->  147
bbox :->  155
bbox :->  161
text
Unspecified
0
0

      
bbox :->  165
bbox :->  147
bbox :->  204
bbox :->  163
text
Unspecified
0
0

      
bbox :->  211
bbox :->  147
bbox :->  221
bbox :->  159
text
Unspecified
0
0

      
bbox :->  229
bbox :->  147
bbox :->  248
bbox :->  162
text
Unspecified
0
0

      
bbox :->  260
bbox :->  147
bbox :->  303
bbox :->  160
text
Unspecified
0
0

      
bbox :->

In [90]:
import cv2

image = cv2.imread(img_file[4])
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) # grayscale
_,thresh = cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV) # threshold
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
dilated = cv2.dilate(thresh,kernel,iterations = 13) # dilate
contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # get contours

# for each contour found, draw a rectangle around it on original image
for contour in contours:
    # get rectangle bounding contour
    [x,y,w,h] = cv2.boundingRect(contour)

    # discard areas that are too large
    if h>300 and w>300:
        continue

    # discard areas that are too small
    if h<40 or w<40:
        continue

    # draw rectangle around contour on original image
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)

# write original image with added contours to disk  
cv2.imshow("contoured.jpg", image)
# cv2.imwrite("contoured.jpg", image)
cv2.waitKey(0)

-1